# Solutions for discussion January
## Single effect blocks for pymc3 and logistic

In [1]:
import pandas as pd, numpy as np
import os
import pyreadr
from dsc.dsc_io import load_rds
cwd = os.path.expanduser("~/GIT/cnv-gene-mapping/data")

In [2]:
beta = pd.read_csv(f"{cwd}/deletion.genes.block20.for_simu.shape0.scale1.beta", header = None, names = ["beta"])

In [3]:
index1 = pd.read_csv(f"{cwd}/deletion.block1.index.csv", header = None, sep = "\t", names = ["id1", "id2"])

In [4]:
index2 = pd.read_csv(f"{cwd}/deletion.block1.forsimu.index.csv", header = None, sep = "\t", names = ["is1", "is2"])

In [5]:
index = pd.concat([index1, index2], axis = 1)

In [6]:
index.tail()

,id1,id2,is1,is2
523,2250,2251,21934,21935
524,2252,2257,21966,21971
525,2258,2260,22009,22011
526,2261,2270,22118,22127
527,2271,2289,22367,22385


In [7]:
index.head()

,id1,id2,is1,is2
0,0,7,30,37
1,8,17,44,53
2,18,22,123,127
3,23,36,194,207
4,37,44,259,266


In [8]:
index["number_effect"] = [np.count_nonzero(beta.loc[item[0]:item[1]]["beta"].tolist()) for idx, item in index2.iterrows()]

In [9]:
idx_beta = index[index["number_effect"] == 1][["is1", "is2"]]

In [10]:
beta_in_block = list()
for idx, item in idx_beta.iterrows():
    beta_in_block.append(beta["beta"][item[0]:item[1]+1].tolist())

In [11]:
index = index[index["number_effect"] == 1][["id1", "id2"]]
index = index.set_index([[i for i in range(index.shape[0])]])

In [12]:
index.shape

(66, 2)

In [13]:
logit_pymc3 = pd.DataFrame()
for idx, item in index.iterrows():
    logit = pyreadr.read_r(f"{cwd}/deletion_simu_20/block_{item[0]}_{item[1]}/deletion.genes.block20.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.logit.rds")
    pymc3 = pd.read_csv(f"{cwd}/deletion_simu_20/block_{item[0]}_{item[1]}/deletion.genes.block20.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.pymc3.gz", sep = "\t")
    pymc3["block"] = f"block{idx}"
    susie = load_rds(f"{cwd}/deletion_simu_20/block_{item[0]}_{item[1]}/deletion.genes.block20.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.SuSiE.L_1.prior_0p005.susie.rds")["pip"]
    res = pd.concat([logit[None][["p1"]], pymc3[["inclusion_probability", "block"]]], axis = 1)
    res["susie"] = susie
    res["beta"] = beta_in_block[idx]
    res = res.rename(columns = {"p1": "logit", "inclusion_probability": "pymc3"})
    logit_pymc3 = pd.concat([logit_pymc3, res])

/home/min/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/min/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [14]:
logit_pymc3["beta"] = logit_pymc3.apply(lambda x: 0 if x["beta"] == -0 else x["beta"], axis = 1)
logit_pymc3["logit"] = [np.round(x, 6) for x in logit_pymc3["logit"]]
logit_pymc3["pymc3"] = [np.round(x, 6) for x in logit_pymc3["pymc3"]]
logit_pymc3["susie"] = [np.round(x, 6) for x in logit_pymc3["susie"]]

In [15]:
logit_pymc3.head()

,logit,pymc3,block,susie,beta
0,0.115353,0.0275,block0,0.0,0.000000
1,0.101018,0.0195,block0,0.0,0.000000
2,0.101018,0.0195,block0,0.0,0.254678
3,0.136522,0.0235,block0,0.0,0.000000
4,0.136522,0.0250,block0,0.0,0.000000


In [16]:
logit_pymc3.to_csv(f"{cwd}/deletion_simu_20/logit_pymc3_comparison.csv", sep = "\t", header = True, index = False)